In [118]:
import pandas as pd
import numpy as np
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] =42
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [119]:
df1 = pd.read_csv("df_all.csv")
del df1['Unnamed: 0']
df1.head()

,Number,ratings2,ratings1,counts2,counts1,dev2,dev1,Text,length,word_leng
0,1,NaN,0.901997,0,32024.0,NaN,5.242998,"A man visits the doctor. The doctor says, I ha...",30,short
1,2,NaN,0.162989,0,35277.0,NaN,5.627972,This couple had an excellent relationship goin...,69,medium
2,3,NaN,0.193411,0,32402.0,NaN,5.448998,Q. Whats 200 feet long and has 4 teeth?A. The ...,17,short
3,4,NaN,-1.412599,0,30512.0,NaN,5.276202,Q. Whats the difference between a man and a to...,21,short
4,5,-2.034618,0.235352,566,73405.0,5.423289,5.302902,"Q. Whats O. J. Simpsons web address?A. Slash, ...",13,very short


In [120]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

vec = TfidfVectorizer(tokenizer=textblob_tokenizer,
                      stop_words='english',
                      max_df =0.9, # if you're in >90%, ignore
                      min_df =0.15, # if you're only in 15%, ignore
                      use_idf=True)

# Fit from the 'text' column of our dataframe
matrix = vec.fit_transform(df1['Text'])

# Then turn it into a new dataframe
results = pd.DataFrame(matrix.toarray(), columns=vec.get_feature_names())

CPU times: user 822 ms, sys: 299 ms, total: 1.12 s
Wall time: 1.43 s


In [121]:
results.head()

,ask,hi,look,man,repli,said,say,thi,wa
0,0.000000,0.000000,0.0,0.842112,0.401275,0.000000,0.360313,0.000000,0.000000
1,0.400093,0.350029,0.0,0.000000,0.409332,0.458754,0.000000,0.419115,0.404648
2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [122]:
%%time
from sklearn.cluster import KMeans

number_of_clusters=5
km = KMeans(n_clusters=number_of_clusters)

print("Fitting", number_of_clusters, "clusters usinga ", matrix.shape, "matrix")

km.fit(matrix)
km.fit

Fitting 5 clusters usinga  (150, 9) matrix
CPU times: user 123 ms, sys: 3.23 ms, total: 126 ms
Wall time: 199 ms


In [123]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vec.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Top terms per cluster:
Cluster 0: look repli ask thi man
Cluster 1: hi look ask wa say
Cluster 2: say thi ask hi repli
Cluster 3: man say ask wa repli
Cluster 4: wa said repli hi ask


In [124]:
df1.Text

0      A man visits the doctor. The doctor says, I ha...
1      This couple had an excellent relationship goin...
2      Q. Whats 200 feet long and has 4 teeth?A. The ...
3      Q. Whats the difference between a man and a to...
4      Q. Whats O. J. Simpsons web address?A. Slash, ...
5      Bill and Hillary Clinton are on a trip back to...
6      How many feminists does it take to screw in a ...
7      Q. Did you hear about the dyslexic devil worsh...
8      A country guy goes into a city bar that has a ...
9      Two cannibals are eating a clown. One turns to...
10     Q. What do a hurricane, a tornado, and a redne...
11     A guy stood over his tee shot for what seemed ...
12     They asked the Japanese visitor if they have e...
13     The father was very anxious to marry off his o...
14     Q:What did the blind person say when given som...
15     Q. What is orange and sounds like a parrot?A. ...
16     How many men does it take to screw in a light ...
17     A dog walks into Western

In [125]:
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

from sklearn.feature_extraction import stop_words
custom_stopwords = ['say', 'hi', 'ask', 'said'] + list(stop_words.ENGLISH_STOP_WORDS)

# Vectorize and save into a new dataframe
vec = TfidfVectorizer(stop_words=custom_stopwords,
                      max_df =0.9, # if you're in >90%, ignore
                      max_features = 1000,
                      use_idf=True)

# Fit from the 'text' column of our dataframe
matrix = vec.fit_transform(df1['Text'])

# Then turn it into a new dataframe
results = pd.DataFrame(matrix.toarray(), columns=vec.get_feature_names())

In [126]:
matrix.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [127]:
%%time
from sklearn.cluster import KMeans

# How many clusters?
number_of_clusters=10
km = KMeans(n_clusters=number_of_clusters)

print("Fitting", number_of_clusters, "clusters usinga ", matrix.shape, "matrix")

# Let's fit it!
km.fit(matrix)
km.fit

Fitting 10 clusters usinga  (150, 1000) matrix
CPU times: user 255 ms, sys: 3.37 ms, total: 258 ms
Wall time: 341 ms


In [128]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vec.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :10]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Top terms per cluster:
Cluster 0: time sasquatch professor students male president student bush negative pig
Cluster 1: 00 replied work 10 common phones asked phone shredder parrot
Cluster 2: asks school teacher replies president mother walks duck tell bartender
Cluster 3: did hear person dyslexic buddhist knock panda hell untie pride
Cluster 4: man doctor news says blah bad room replies cancer dont
Cluster 5: lightbulb does change darwin takes teddy oh programmers problem eating
Cluster 6: says im father guy goes little priest time married man
Cluster 7: bulb light screw men does woman floor satisfy radio wants
Cluster 8: whats difference slash used toilet plumber great government stallone sketch
Cluster 9: engineer just company mechanical hillary engineers look room electrical software


In [129]:
km.labels_

array([4, 1, 8, 8, 8, 9, 7, 3, 6, 5, 1, 1, 1, 7, 3, 1, 7, 6, 1, 8, 8, 2, 1,
       1, 8, 1, 4, 9, 7, 8, 2, 4, 4, 2, 6, 6, 2, 2, 7, 5, 5, 7, 8, 2, 2, 6,
       9, 2, 9, 6, 3, 1, 6, 5, 2, 4, 1, 5, 1, 3, 9, 9, 9, 3, 4, 9, 6, 4, 2,
       6, 9, 0, 8, 7, 0, 4, 1, 8, 1, 6, 4, 6, 4, 9, 7, 2, 4, 4, 5, 5, 3, 2,
       9, 6, 9, 1, 0, 6, 1, 8, 3, 6, 4, 0, 6, 9, 1, 6, 1, 0, 1, 8, 1, 2, 7,
       9, 1, 4, 2, 1, 6, 1, 0, 3, 1, 9, 6, 1, 7, 4, 9, 6, 1, 4, 4, 8, 0, 0,
       0, 6, 1, 6, 6, 0, 0, 1, 0, 2, 6, 2], dtype=int32)

In [130]:
df1['group'] = pd.Series(km.labels_)

In [131]:
df1

,Number,ratings2,ratings1,counts2,counts1,dev2,dev1,Text,length,word_leng,group
0,1,NaN,0.901997,0,32024.0,NaN,5.242998,"A man visits the doctor. The doctor says, I ha...",30,short,4
1,2,NaN,0.162989,0,35277.0,NaN,5.627972,This couple had an excellent relationship goin...,69,medium,1
2,3,NaN,0.193411,0,32402.0,NaN,5.448998,Q. Whats 200 feet long and has 4 teeth?A. The ...,17,short,8
3,4,NaN,-1.412599,0,30512.0,NaN,5.276202,Q. Whats the difference between a man and a to...,21,short,8
4,5,-2.034618,0.235352,566,73405.0,5.423289,5.302902,"Q. Whats O. J. Simpsons web address?A. Slash, ...",13,very short,8
5,6,NaN,1.330277,0,39599.0,NaN,4.976994,Bill and Hillary Clinton are on a trip back to...,121,long,9
6,7,-1.952510,-0.633947,50692,73401.0,5.370893,5.610902,How many feminists does it take to screw in a ...,14,very short,7
7,8,-0.716500,-0.996267,50692,73411.0,5.153371,4.968263,Q. Did you hear about the dyslexic devil worsh...,15,short,3
8,9,NaN,-0.620039,0,30741.0,NaN,5.201667,A country guy goes into a city bar that has a ...,91,long,6
9,10,NaN,1.181706,0,39352.0,NaN,5.183989,Two cannibals are eating a clown. One turns to...,19,short,5


In [132]:
df1.to_csv("df_groups.csv")

In [133]:
df1.group.value_counts().sum()

150

In [134]:
df1.group.value_counts()

1    28
6    23
4    17
9    16
2    16
8    13
0    12
7    10
3     8
5     7
Name: group, dtype: int64

In [135]:
df1.groupby('group').ratings1.mean().sort_values(ascending=False)

group
4    1.780414
9    1.405766
0    1.224795
6    1.121128
2    0.713412
5    0.608206
7    0.491643
8    0.251821
1   -0.166297
3   -0.523707
Name: ratings1, dtype: float64

From best to worst rating:
Cluster 0: head line mickey tornado redneck time divorce 14 13 began
Cluster 8: woman man lost minutes beautiful pants ages atom watson time
Cluster 6: father school im little boy teacher goes man son mother
Cluster 9: doctor man news says blah bad replies neutron cancer dont
Cluster 7: engineer person did hell 10 just guy says parrot opposite
Cluster 1: room st peter lightbulb president phones languages just american phone
Cluster 3: difference whats slash sasquatch toilet engineers plumber government used sketch
Cluster 2: says does br asks darwin long turns duck hillary eating
Cluster 5: new work shredder replied come 00 asked station need japanese
Cluster 4: bulb light screw does men radio thats funny night presidents

In [136]:
df_group1 = df1.loc[df1['group'] == 0]
df_group2 = df1.loc[df1['group'] == 1]
df_group3 = df1.loc[df1['group'] == 2]
df_group4 = df1.loc[df1['group'] == 3]
df_group5 = df1.loc[df1['group'] == 4]
df_group6 = df1.loc[df1['group'] == 5]
df_group7 = df1.loc[df1['group'] == 6]
df_group8 = df1.loc[df1['group'] == 7]
df_group9 = df1.loc[df1['group'] == 8]
df_group10 = df1.loc[df1['group'] == 9]

In [137]:
df_group1.Text

71     On the first day of college, the Dean addresse...
74     Q:Do you know the difference between an intell...
96     A teacher is explaining to her class how diffe...
103    As a pre-med student, I had to take a difficul...
109    One day, a professor was giving a big test to ...
122    When most people claim to be killing time, its...
136    Deep within a forest, a little turtle began to...
137    WASHINGTON (Reuters) - A tragic fire on Monday...
138    In a Veterans Day speech, President Bush vowed...
143    A man is driving in the country one evening wh...
144    A blonde, brunette, and a red head are all lin...
146    It was the day of the big sale. Rumors of the ...
Name: Text, dtype: object

In [138]:
df_group1.ratings2.sort_values()

74    -0.859899
122   -0.233593
138    1.596541
136    1.730777
146    1.782721
143    1.868181
144    2.458938
109    2.771443
137    2.799709
96     2.907296
103    3.422973
71     3.505117
Name: ratings2, dtype: float64

In [139]:
df_group2.Text

1      This couple had an excellent relationship goin...
10     Q. What do a hurricane, a tornado, and a redne...
11     A guy stood over his tee shot for what seemed ...
12     They asked the Japanese visitor if they have e...
15     Q. What is orange and sounds like a parrot?A. ...
18     Q:If a person who speaks three languages is ca...
22     Q:What is the Australian word for a boomerang ...
23     What do you get when you run over a parakeet w...
25     A guy walks into a bar and sits down next to a...
51     Q:What do Monica Lewinsky and Bob Dole have in...
56     Why are there so many Joness in the phone book...
58     The Chukcha (Russian Eskimo) phones up the Rus...
76     If pro- is the opposite of con-, then congress...
78     Q:Ever wonder why the IRS calls it Form 1040?A...
95     Two attorneys went into a diner and ordered tw...
98     A bus station is where a bus stops.A train sta...
106    (A) The Japanese eat very little fat and suffe...
108    Do you believe in life a

In [140]:
df_group2.ratings2

1           NaN
10          NaN
11          NaN
12    -0.649263
15    -1.630446
18     0.774643
22     1.081558
23    -1.057448
25     2.515128
51     0.093189
56    -0.480900
58     1.278032
76     1.550592
78     0.296158
95     2.317327
98     1.939704
106    2.585249
108    2.086600
110    2.801502
112    1.437430
116    3.212989
119    1.616199
121    2.568052
124    2.820173
127    2.589191
132    1.443055
140   -2.748204
145    0.176494
Name: ratings2, dtype: float64

In [141]:
df_group3.Text

21     A duck walks into a pharmacy and asks for a co...
30     President Clinton looks up from his desk in th...
33     Out in the backwoods of some midwestern state,...
36     A Jewish young man was seeing a psychiatrist f...
37     May I take your order? the waiter asks.Yes, ho...
43     A horse walks into a bar. The bartender asks:b...
44     A boy comes home from school and tells his mot...
47     The graduate with a Science degree asks, Why d...
54     A woman has twins, and gives them up for adopt...
68     This guys wife asks, Honey, if I died would yo...
85     A neutron walks into a bar and orders a drink....
91     Early one morning a mother went to her sleepin...
113    Sherlock Holmes and Dr. Watson go on a camping...
118    One day the first grade teacher was reading th...
147    Recently a teacher, a garbage collector, and a...
149    In an interview with David Letterman, Carter p...
Name: Text, dtype: object

In [142]:
df_group3.ratings2

21     1.284563
30     2.372554
33     0.725621
36     0.159677
37     1.834387
43    -0.985106
44     1.586318
47     2.731921
54     0.452153
68     3.249782
85     1.521713
91     1.985260
113    3.192744
118    2.797977
147    3.061374
149    2.810493
Name: ratings2, dtype: float64

In [143]:
df_group4.Text

7      Q. Did you hear about the dyslexic devil worsh...
14     Q:What did the blind person say when given som...
50     Did you hear that Clinton has announced that t...
59     What did the Buddhist say to the hot dog vendo...
63     What is the rallying cry of the International ...
90     A Panda walks into a bar, sits down at a table...
100    Did you hear about the Buddhist who refused No...
123    Person 1:Hey, wanna hear a great knock-knock j...
Name: Text, dtype: object

In [144]:
df_group4.ratings2.sort_values()

123   -2.161861
14    -1.430683
7     -0.716500
50    -0.447101
100   -0.242302
63    -0.080515
59     1.087394
90     1.963375
Name: ratings2, dtype: float64

In [145]:
df_group5.Text

0      A man visits the doctor. The doctor says, I ha...
26     Bill Clinton returns from a vacation in Arkans...
31     A man arrives at the gates of heaven. St. Pete...
32     What do you call an American in the finals of ...
55     A man and Cindy Crawford get stranded on a des...
64     Two rednecks were seated at the end of a bar w...
67     A man piloting a hot air balloon discovers he ...
75     There once was a man and a woman that both got...
80     An Asian man goes into a New York CityBank to ...
82     What a woman says:This place is a mess! Cmon,y...
86     A man who recently completed a routine physica...
87     A Czechoslovakian man felt his eyesight was gr...
102    Nurse:Doctor, Doctor, theres an invisible man ...
117    A man goes into a drug store and asks the phar...
129    An old man goes to the doctor for his yearly p...
133    An artist asked the gallery owner if there had...
134    A guy walked past a mental hospital and heard ...
Name: Text, dtype: object

In [146]:
df_group5.ratings2.sort_values()

32    -0.184266
102    0.812150
129    1.469692
134    1.960441
117    1.983338
80     2.104749
87     2.307332
86     2.412386
82     2.467059
55     2.489225
64     2.542890
75     2.922367
26     2.933849
133    3.174689
67     3.386933
31     3.504556
0           NaN
Name: ratings2, dtype: float64

In [147]:
df_group6.Text

9     Two cannibals are eating a clown. One turns to...
39    How many Irishmen does it take to change a lig...
40    What does an atheist say during an orgasm?Oh D...
53    The Pope dies and, naturally, goes to heaven. ...
57    How many teddy bears does it take to change a ...
88    A radio conversation between a US naval ship a...
89    Q:How many programmers does it take to change ...
Name: Text, dtype: object

In [148]:
df_group6.ratings2.sort_values()

57   -1.391845
40    0.082778
39    1.514314
89    1.733508
53    2.948606
88    3.606178
9          NaN
Name: ratings2, dtype: float64

In [149]:
df_group7.Text

8      A country guy goes into a city bar that has a ...
17     A dog walks into Western Union and asks the cl...
34     An explorer in the deepest Amazon suddenly fin...
35     A guy walks into a bar, orders a beer and says...
45     A couple has been married for 75 years. For th...
49     A guy goes into confession and says to the pri...
52     One Sunday morning William burst into the livi...
66     Once upon a time, two brooms fell in love and ...
69     Employer to applicant:In this job we need some...
79     Hillary, Bill Clinton and the Pope are sitting...
81     Q:How do you keep a computer programmer in the...
93     Two atoms are walking down the street when one...
97     Age and Womanhood: 1. Between the ages of 13 a...
101    A man escaped jail by digging a hole from his ...
104    A couple of hunters are out in the woods in th...
107    A man approached a very beautiful woman in a l...
120    A drunk staggers into a Catholic Church, enter...
126    A little boy goes to his

In [150]:
df_group7.ratings2.sort_values()

17    -0.127532
66    -0.035075
139    0.631518
101    1.158694
81     1.253061
45     1.318530
69     1.686612
93     1.940943
141    2.002585
79     2.236901
97     2.255475
120    2.370295
148    2.399215
142    2.718819
35     2.924089
126    3.174746
131    3.247761
107    3.260290
49     3.273599
34     3.560140
104    3.711070
52     3.714249
8           NaN
Name: ratings2, dtype: float64

In [151]:
df_group8.Text

6      How many feminists does it take to screw in a ...
13     The father was very anxious to marry off his o...
16     How many men does it take to screw in a light ...
28     An old Scotsman is sitting with a younger Scot...
38     What is the difference between men and women?A...
41     Two men are discussing the age old question:wh...
73     Q:How many stalkers does it take to change a l...
84     Q:How many presidents does it take to screw in...
114    A lady bought a new Lexus. It cost a bundle. T...
128    A group of girlfriends is on vacation when the...
Name: Text, dtype: object

In [152]:
df_group8.ratings2.sort_values()

6     -1.952510
73    -0.604289
84     0.282514
16     0.674613
41     1.596508
114    1.626380
38     1.737126
28     3.045842
128    3.583135
13          NaN
Name: ratings2, dtype: float64

In [153]:
df_group9.Text

2      Q. Whats 200 feet long and has 4 teeth?A. The ...
3      Q. Whats the difference between a man and a to...
4      Q. Whats O. J. Simpsons web address?A. Slash, ...
19     Whats the difference between a Macintosh and a...
20     Whats the difference between a used tire and 3...
24     Two kindergarten girls were talking outside:on...
29     Q:Whats the difference between a lawyer and a ...
42     Arnold Schwarzenegger and Sylvester Stallone a...
72     Q:What is the difference between George Washin...
77     Q:Whats the difference between the government ...
99     Q:Whats the difference between greeting a quee...
111    When my three-year-old son opened the birthday...
135    A man went to apply for a job. After filling o...
Name: Text, dtype: object

In [154]:
df_group9.ratings2.sort_values()

4     -2.034618
19    -0.702816
42     0.301393
135    0.461225
99     1.040923
29     1.048881
24     1.202782
72     1.754678
111    2.259419
77     2.267950
20     2.989191
2           NaN
3           NaN
Name: ratings2, dtype: float64

In [155]:
df_group10.Text

5      Bill and Hillary Clinton are on a trip back to...
27     A mechanical engineer, an electrical engineer,...
46     There was an engineer who had an exceptional g...
48     Three engineering students were gathered toget...
60     During a recent publicity outing, Hillary snea...
61     A group of managers were given the assignment ...
62     An engineer, a physicist and a mathematician a...
65     A lawyer opened the door of his BMW, when sudd...
70     At a recent Sacramento PC Users Group meeting,...
83     Q:What is the difference between mechanical en...
92     Reaching the end of a job interview, the human...
94     Just a thought.Before criticizing someone, wal...
105    An engineer dies and reports to the pearly gat...
115    A man in a hot air balloon realized he was los...
125    A Briton, a Frenchman and a Russian are viewin...
130    A guy had been feeling down for so long that h...
Name: Text, dtype: object

In [156]:
df_group10.ratings2.sort_values()

130    0.743464
70     1.092254
94     1.195465
115    1.747176
60     1.891188
27     2.150815
83     2.151729
65     2.793044
92     2.798529
46     2.987722
62     2.994831
48     3.090087
125    3.186395
61     3.310631
105    3.337722
5           NaN
Name: ratings2, dtype: float64